# Описание задания

Перед выполнением задания установите Jupyter Notebook или используйте Google Colab.
Файл с исходными данными: web_clients_correct.csv

## Общие требования к программе:

- код должен быть грамотно декомпозирован: каждая функция отвечает за свою конкретную задачу, дублирующиеся функции переиспользуются, а их код не повторяется;
- в коде отсутствуют глобальные переменные.

Цель: разработать систему, которая будет формировать описания покупателей на основе характеристик, представленных в файле.

На входе — CSV-файл с характеристиками покупателей.

На выходе — TXT-файл с описаниями.

## Шаги:

1. Загрузить CSV-файл.
2. Выполнить парсинг (извлечение атрибутов из структурированного текста).
3. Преобразовать данные (при необходимости).
4. Сформировать текстовое описание по шаблону.
5. Записать в единый TXT-файл.

## Пример строки на входе:
```
ФИО: Allen Miss. Elisabeth Walton
Пол: female
Возраст: 29
Устройство, с которого выполнялась покупка: mobile
Браузер: Chrome
Сумма чека: 885
Регион покупки: St Louis: MO
```

## Пример строки на выходе:
```
Пользователь Allen Miss. Elisabeth Walton женского пола, 29 лет совершила покупку на 885 у.е. с мобильного браузера Chrome. Регион, из которого совершалась покупка: St Louis: MO.
```

In [ ]:
from dataclasses import dataclass
import os
import csv


class Util:
    """
    Various utilities methods
    """
    @staticmethod
    def declension(number: int, words: tuple) -> str:
        """
        Declension of a word depending on the number.
        """
        if len(words) != 3:
            raise RuntimeError('Count of words must be 3')
        remainder = number % 10
        if number == 0 or remainder == 0 or remainder >= 5 or number in range(11, 19):  
            st = str(number), words[0]
        elif remainder == 1:  
            st = str(number), words[1]      
        else:  
            st = str(number), words[2]
        return " ".join(st)


@dataclass
class Customer():
    """
    Customer data class
    """
    name: str
    gender: str
    age: int
    device: str
    browser: str
    receipt_sum: int
    region: str
    __template__: str = "Пользователь {name} {gender_repr}, {age_repr} совершила покупку на {receipt_sum} у.е. с {device_repr} браузера {browser}. Регион, из которого совершалась покупка: {region}."
    __devices_repr__ = {
        'desktop': 'настольного',
        'mobile': 'мобильного',
        'tablet': 'планшетного',
        'laptop': 'ноутбучного',
    }
    __genders_repr__ = {
        'male': 'мужского пола',
        'female': 'женского пола',
    }

    def get_repr(self, data_type, data, key) -> str:
        if key in data:
            return data[key]
        else:
            raise RuntimeError("Unknown {}: {}".format(data_type, key))

    def get_device_repr(self) -> str:
        return self.get_repr('device', self.__devices_repr__, self.device)

    def get_gender_repr(self) -> str:
        return self.get_repr('gender', self.__genders_repr__, self.gender)
        
    def __str__(self) -> str:
        params = {
            'name': self.name,
            'gender_repr': self.get_gender_repr(),
            'age_repr': Util.declension(self.age, ('лет', 'год', 'года')),
            'receipt_sum': self.receipt_sum,
            'device_repr': self.get_device_repr(),
            'browser': self.browser,
            'region': self.region,
        }
        return self.__template__.format(**params)


def main() -> None:
    file_name = input('Введите имя файла: ')
    output_name = input('Введите имя выходного файла: ')
    if not os.path.exists(file_name):
        print('file {} dont exist.'.format(file_name))
        return
    with open(file_name, 'r', encoding='utf8') as fp_in:
        with open(output_name, 'w', encoding='utf8') as fp_out:
            reader = csv.reader(fp_in)
            for idx, line in enumerate(reader):
                if idx == 0:
                    continue

                try:
                    name, device, browser, gender, age, summary, region = line
                    customer = Customer(
                        name=name, 
                        gender=gender, 
                        age=int(age), 
                        device=device, 
                        browser=browser, 
                        receipt_sum=int(summary),
                        region=region
                    )
                    out_line = "{}\n".format(str(customer))
                    fp_out.write(out_line)
                except Exception as e:
                    print("Error: Incorrect line '{}' with message '{}'".format(", ".join(line), str(e)))


if __name__ == '__main__':
    main()
